# Take-Home Challenge 2: Relax Data Science Challenge

### In this assignment we will look to review the assigned csv files, looking out for any missing data values and interpreting accordingly.  We will then join the files together and run a single model not necessarily for its output (as we usually would) but so that we can extract the best features that the model is identifying with for this exercise.  The choice of model will be determined once we start to go through the data a bit and see what our expected target is.

#### As per usual cadence let's read in our csv files and review them, as well as of course reading in some provisionary libraries to start with to get us on our way.  We will update this library list as we move along; I will just update accodingly up top so its a nice organized list.

In [1]:
# First let's import our libraries to start.
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
# Now let's read in our two csv files for this exercise.  Note they are local in our directory, in a real-world scenario they won't be and would require some extra coding to get them to read in.
users_df = pd.read_csv('takehome_users.csv', encoding = 'ISO-8859-1')
users_df.head()
# Slight curveball there, there was formatting that we needed to research and get through.  All fixed!


,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [3]:
# Now let's read in the other one.
engage_df = pd.read_csv('takehome_user_engagement.csv')
engage_df.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


#### Great now let's do some quick summaries on our datasets to get an idea on each one before we continue.

In [4]:

# Let's do the users df first.
users_df.shape

(12000, 10)

In [5]:
users_df.columns

Index(['object_id', 'creation_time', 'name', 'email', 'creation_source',
       'last_session_creation_time', 'opted_in_to_mailing_list',
       'enabled_for_marketing_drip', 'org_id', 'invited_by_user_id'],
      dtype='object')

In [ ]:
users_df.describe()

,object_id,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
count,12000.00000,8.823000e+03,12000.000000,12000.000000,12000.000000,6417.000000
mean,6000.50000,1.379279e+09,0.249500,0.149333,141.884583,5962.957145
std,3464.24595,1.953116e+07,0.432742,0.356432,124.056723,3383.761968
min,1.00000,1.338452e+09,0.000000,0.000000,0.000000,3.000000
25%,3000.75000,1.363195e+09,0.000000,0.000000,29.000000,3058.000000
50%,6000.50000,1.382888e+09,0.000000,0.000000,108.000000,5954.000000
75%,9000.25000,1.398443e+09,0.000000,0.000000,238.250000,8817.000000
max,12000.00000,1.402067e+09,1.000000,1.000000,416.000000,11999.000000


In [7]:
users_df.isna().sum()

object_id                        0
creation_time                    0
name                             0
email                            0
creation_source                  0
last_session_creation_time    3177
opted_in_to_mailing_list         0
enabled_for_marketing_drip       0
org_id                           0
invited_by_user_id            5583
dtype: int64

#### Wow that's quite a few missing values in those two features for last_session_creation_time and invited_by_user_id.  Before we go into filling and/or imputation of those values let's look at the other dataset first to see if there's any connection to those missing values or a reason why.

In [8]:
engage_df.shape

(207917, 3)

In [9]:
engage_df.columns

Index(['time_stamp', 'user_id', 'visited'], dtype='object')

#### Ah I see the user_id there can be our merging key once renamed to object_id (or vice versa) to the other dataset.  Good stuff there, let's keep moving.

In [ ]:
engage_df.describe()

,user_id,visited
count,207917.000000,207917.0
mean,5913.314197,1.0
std,3394.941674,0.0
min,1.000000,1.0
25%,3087.000000,1.0
50%,5682.000000,1.0
75%,8944.000000,1.0
max,12000.000000,1.0


In [ ]:
engage_df.isna().sum()

time_stamp    0
user_id       0
visited       0
dtype: int64

#### Great there are no missing values.  Before we perform our rename and merge though let's take care of the missing values from our users df.

#### So since we are dealing with user data here let's consider why there might be missing values for both of these features in the users_df dataset (last_session_creation_time and invited_by_user_id).  Without getting too deep in the weeds about it, the reason(s) could be as explanatory as new users signing up then never logging in and/or users signing up on their own without being invited by another user.  These are just examples.  Either way since both of these features are tied to user data I don't want to use normal methods such as forward/back fill and linear imputation, as it would use surrounding existing values to fill in the na values.  This would not be ideal.  Instead we will just fill all the na values with a "0" and move forward.

In [12]:
users_df['last_session_creation_time'] = users_df['last_session_creation_time'].fillna(0)
users_df['invited_by_user_id'] = users_df['invited_by_user_id'].fillna(0)

# Check your work.
users_df.isna().sum()

object_id                     0
creation_time                 0
name                          0
email                         0
creation_source               0
last_session_creation_time    0
opted_in_to_mailing_list      0
enabled_for_marketing_drip    0
org_id                        0
invited_by_user_id            0
dtype: int64

#### Good, now let's rename the object_id feature in the engage_df dataset and conduct the join so we have a combined df to work with for our forthcoming model.

In [ ]:
users_df = users_df.rename(columns = {'object_id' : 'user_id'})
users_df.head()

,user_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


#### Not the most important thing, but I am going to strip the time of day from both creation_time from the users_df and time_stamp from our engage_df.  This will make it a lot easier to read and process.  What is important that we need to take care of though is that last_session_creation_time.  As it is very unreadable for us in its present format we will need to convert it using datetime.

In [14]:
users_df['creation_time'] = pd.to_datetime(users_df['creation_time'], errors = 'coerce').dt.normalize()
users_df['last_session_creation_time'] = pd.to_datetime(users_df['last_session_creation_time'], unit = 's', errors = 'coerce').dt.normalize()
users_df.head()

,user_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-22,1,0,11,10803.0
1,2,2013-11-15,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-31,0,0,1,316.0
2,3,2013-03-19,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,2013-03-19,0,0,94,1525.0
3,4,2013-05-21,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,2013-05-22,0,0,1,5151.0
4,5,2013-01-17,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,2013-01-22,0,0,193,5240.0


In [15]:
engage_df['time_stamp'] = pd.to_datetime(engage_df['time_stamp'], errors = 'coerce').dt.normalize()
engage_df.head()

,time_stamp,user_id,visited
0,2014-04-22,1,1
1,2013-11-15,2,1
2,2013-11-29,2,1
3,2013-12-09,2,1
4,2013-12-25,2,1


#### Alright now let's merge our two datasets together.

In [16]:
combined_df = pd.merge(users_df, engage_df, on = 'user_id', how = 'left')
combined_df.sample(10)

,user_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,time_stamp,visited
208239,11815,2012-10-14,Araujo Yasmin,YasminBarbosaAraujo@jourrapide.com,ORG_INVITE,2012-10-15,1,0,206,6035.0,2012-10-15,1.0
114724,6282,2012-08-04,Andreasen Anne-Lise,Anne-LiseMAndreasen@jourrapide.com,ORG_INVITE,2014-06-03,1,1,106,4713.0,2014-01-24,1.0
27409,1693,2012-05-31,Faulkner Hayden,HaydenFaulkner@gmail.com,SIGNUP_GOOGLE_AUTH,2014-05-12,0,1,50,0.0,2013-06-23,1.0
38186,2300,2012-09-24,Mills Isabelle,IsabelleMills@jourrapide.com,SIGNUP,2014-06-01,0,0,370,0.0,2013-10-10,1.0
34261,2042,2012-06-10,Silva Davi,DaviRodriguesSilva@gmail.com,SIGNUP_GOOGLE_AUTH,2014-06-04,0,0,129,0.0,2014-01-23,1.0
146197,8259,2012-06-05,Wechsler Karin,KarinWechsler@gmail.com,ORG_INVITE,2014-06-02,0,0,160,3885.0,2012-08-23,1.0
81513,4472,2012-11-28,Schweitzer Nadine,NadineSchweitzer@gmail.com,SIGNUP,2014-06-04,0,0,353,0.0,2013-11-25,1.0
176200,9747,2013-09-09,Delacruz Linda,LindaJDelacruz@yahoo.com,ORG_INVITE,2014-06-04,0,0,34,8657.0,2014-05-03,1.0
169362,9479,2013-11-27,Sharp Toby,TobySharp@gmail.com,GUEST_INVITE,2014-06-04,0,0,15,6506.0,2014-04-09,1.0
116319,6370,2012-06-21,Rocha Davi,DaviPintoRocha@gmail.com,SIGNUP_GOOGLE_AUTH,2014-05-22,1,1,217,0.0,2012-11-03,1.0


#### As general housekeeping let's do a NA check.

In [17]:
combined_df.isna().sum()


user_id                          0
creation_time                    0
name                             0
email                            0
creation_source                  0
last_session_creation_time       0
opted_in_to_mailing_list         0
enabled_for_marketing_drip       0
org_id                           0
invited_by_user_id               0
time_stamp                    3177
visited                       3177
dtype: int64

#### Glad we checked!  Similar to what we did above, we will issue "0" to those values to be consistent.

In [18]:
combined_df['time_stamp'] = combined_df['time_stamp'].fillna(0)
combined_df['visited'] = combined_df['visited'].fillna(0)

# Let's check our work to make sure we were successful.
combined_df.isna().sum()

user_id                       0
creation_time                 0
name                          0
email                         0
creation_source               0
last_session_creation_time    0
opted_in_to_mailing_list      0
enabled_for_marketing_drip    0
org_id                        0
invited_by_user_id            0
time_stamp                    0
visited                       0
dtype: int64

#### Okay now we are getting to the point of this exercise.  Let's review the key instructions:
Defining   an    "adopted   user"    as   a   user   who     has   logged   into   the   product   on   three   separate 
days   in   at   least   one   seven-­day period
#### So it looks like we need to create a feature here for this instruction, and now I am starting to see what type of model we may need.  Once we create the feature for our adopted_users, we want to classify each user as an adopted_user or not based on this denoted activity.  Sounds like Logistic Regression to me!

#### The fun part though is creating this target feature in adopted_user.  We need to show, in at least 1 7-day period (which means we need to check all 7 day periods), users logging in at least 3 times.  I am thinking 7 day rolling windows here.

In [19]:
# Side quest here, for some reason time_stamp didn't convert over to datetime type.  Fixing now.
combined_df['time_stamp'] = pd.to_datetime(combined_df['time_stamp'], errors='coerce')


In [20]:
# A quick type check and conversion
combined_df['invited_by_user_id'] = combined_df['invited_by_user_id'].astype('int64')
combined_df['visited'] = combined_df['visited'].astype('int64')
combined_df.dtypes

user_id                                int64
creation_time                 datetime64[ns]
name                                  object
email                                 object
creation_source                       object
last_session_creation_time    datetime64[ns]
opted_in_to_mailing_list               int64
enabled_for_marketing_drip             int64
org_id                                 int64
invited_by_user_id                     int64
time_stamp                    datetime64[ns]
visited                                int64
dtype: object

In [21]:
# We have a little bit of prep work to do before we just create the feature.  Let's sort and drop duplicates for our features we will use for this.
combined_df = combined_df.sort_values(by = ['user_id', 'time_stamp'])
combined_df = combined_df.drop_duplicates(subset = ['user_id', 'time_stamp'])


In [25]:
# Now we will make a quick function for grouping by user and applying our desired 7-day rolling window.
def is_adopted_user(group):
    # Apply the 7-day rolling window
    group = group.set_index('time_stamp')
    group['count'] = 1
    rolling_sum = group['count'].rolling('7D').sum()
    
    # Then condition this if the user logged on at least 3x within ANY 7-day window.
    return (rolling_sum >= 3).any()

# Now apply to our users
adopted_users = combined_df.groupby('user_id', group_keys=False).apply(is_adopted_user).astype('int64')

# Now create and merge our work to establish our target
combined_df['adopted_user'] = combined_df['user_id'].map(adopted_users)

print(combined_df['adopted_user'].value_counts())


adopted_user
1    197918
0     13176
Name: count, dtype: int64


C:\Users\ryanm\AppData\Local\Temp\ipykernel_17288\3769809681.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  adopted_users = combined_df.groupby('user_id', group_keys=False).apply(is_adopted_user).astype('int64')


#### That was a little tricky but we got the result we were looking for.  Now on to the last part of this exercise.  We will run a basic Logistic Regression model using our newly developed adopted_user target feature and for the X features just include the remaining int type features.  We no longer will need the datetime or object features at this point, so that will narrow down our focus for the model.

In [28]:
# Establish our X and y variables for the model.
X = combined_df.select_dtypes(include = ['int64']).drop(columns = ['adopted_user'])
y = combined_df['adopted_user']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

#### Good now let's run the model, and note our objective isn't the end metrics its to look at and extract the best predictive features from our fit model here.  So we won't even be looking at metrics and just be using base parameters in this notebook.

In [29]:
log_reg = LogisticRegression(max_iter = 1000)
log_reg.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

#### Great now let's extract and review our coefficients and feature importances.

In [30]:
# Access the coefficients of the model
coefficients = log_reg.coef_[0]

# Create a separate dataframe to view the coefficients, in the form of feature importances
feature_importances = pd.DataFrame({
    'Feature' : X.columns,
    'Coefficient' : coefficients
})

# We will add a column to our custom dataframe for ranking or sorting the coefficients
feature_importances['Abs_Coefficient'] = feature_importances['Coefficient'].abs()

# Now sort by the most important features
feature_importances = feature_importances.sort_values(by = 'Abs_Coefficient', ascending = False)

print(feature_importances)

                      Feature  Coefficient  Abs_Coefficient
5                     visited     8.663043         8.663043
2  enabled_for_marketing_drip    -0.064142         0.064142
1    opted_in_to_mailing_list     0.059584         0.059584
3                      org_id     0.001586         0.001586
4          invited_by_user_id     0.000008         0.000008
0                     user_id    -0.000005         0.000005


## Summary Results and Conclusions

#### In this exercise we were tasked with reviewing two datasets and determining what features are contributing the most to this data.  We cleaned the data and reformatted it, merged it together to have one unified dataset, then ran a (very) basic Logistic Regression model on it with our newly-created target of 'adopted_user', which was part of the itinerary of the exercise.  After the model was fit we then looked to extract the coefficients based on their importance.

#### Referencing the output in the above cell you can clearly see that 'visited' is by far the dominant feature based on coefficient score.  It falls off sharply after this, with the rest of the features contributing very little or even negatively.  With having one feature contributing so much this can actually be a detriment in some ways, and further investigation may be necessary.  I would recommend (if further work would be performed with this data) that another model be ran using a regularization method such as Lasso or preferably Ridge.  You could also scale the data first and then re-run and model instances, as we did not scale data here for this exercise.  Also since we stayed with an extremely basic model, you can explore with a much more robust set of parameters to explore and see if that changes the coefficient output any. A third model could even be performed removing the visited feature completely to see how it effects the remaining features since this feature casts such a large shadow over the others (just considering feature performance here not model output).